In [1]:
# import db_conn
import pandas as pd
import re
pd.options.display.max_columns = None

In [2]:
# conn = db_conn.get_connection()

## 1. 추출된 전체 부작용
추출된 전체 부작용 중 식별이 가능한 UNIQUE한 약물만을 추려냈습니다.

In [3]:
def get_total_ads():
    cur = conn.cursor()
    cur.execute("""
    SELECT DISTINCT(TRIM(LOWER(d.tt_ad))) AS t_ad
    FROM
    (SELECT id, tt_ad
    FROM article_table_sentences_m M,
        UNNEST(STRING_TO_ARRAY(t_ad, ' , ')) AS tt_ad
    WHERE (tt_ad='') is false) as d;
    """)
    return cur.fetchall()

def get_ad_dictionary():
    cur = conn.cursor()
    cur.execute("""
    SELECT DISTINCT(TRIM(LOWER(llt_name))) AS t_ad
    FROM meddra_llt_181022 M;
    """)
    return cur.fetchall()

In [4]:
# ads = pd.DataFrame(get_total_ads())
ads = pd.read_csv('./distinct_ads.tsv', sep='\t')

In [5]:
# ads.to_csv('./ads.csv', sep=',')

In [6]:
ads.columns=['name']

In [7]:
ads['refined'] = ''
ads.head()

,name,refined
0,anuloaortic ectasia,
1,mass-cavity,
2,presyncope,
3,seizure,
4,aphthous ulcer,


#### 전처리 전 총 부작용의 수는 9282개 입니다

In [8]:
ads.shape

(9282, 2)

### 2. 전처리

#### A. dictionary를 참고하여, 정확히 일치하는 부작용 명은 전처리 없이 포함합니다.

In [9]:
# dictionary = get_ad_dictionary()
dictionary = pd.read_csv('./dictionary_ads.tsv', sep='\t')
# dictionary = list(map(lambda x: str(x[0]).lower().replace('"', ''), dictionary))

In [10]:
len(dictionary)

60939

In [11]:
dictionary[:10]

,t_ad
0,ventilation pneumonitis
1,11-beta-hydroxylase deficiency
2,11-oxysteroid activity incr
3,11-oxysteroid activity increased
4,17 ketosteroids urine abnormal
5,17 ketosteroids urine abnormal nos
6,17 ketosteroids urine decreased
7,17 ketosteroids urine high
8,17 ketosteroids urine increased
9,17 ketosteroids urine low


In [13]:
# ads.sort_values(by=['name'])[350:400]

#### A. 부작용을 포함하고 있는 구절이 추출 된 경우, 부작용 외적인 단어들을 제거합니다

In [14]:
replace_with_space = ['<70%', '%', 'values)', '(', ')', '-1987', '-1993', ':', '±', '↑', 'australia-new',
                     'scale-7', 'c-statistics', '-defined', '-derived', '-like', 'like', 'non-beverage', 'beverage',
                     'scale-0', '-days', 'fluorine-18-labelled', '-specific', 'co-variate', '-guided', '≥2°',
                     '≥1', 'composting', 'daids', 'decentralised', 'dmards', 'forage', 'full-scale', 'glues',
                     'granger', 'greyscale', 'grey-scale', 'hanging', 'harvesting', 'immersed', 'invariably',
                     '-guidance', 'lifelon', 'mwaura', 'naito', 'nesting', 'nomad', 'nonsystemi', '≥60', 'primers',
                     'prohibited', 'sanger', '-1993', '<50']

replace_ends_with = ['-related']

In [ ]:
rep = dict((re.escape(k), ' ') for k in replace_with_space)
pattern = re.compile("|".join(rep.keys()))

for ind, row in ads.iterrows():
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if refined.strip() == '':
        ads.drop(ind, inplace=True)
    elif row['name'].strip() != refined.strip():
        ads.loc[ind]['refined'] = refined
        print(row['name'])
        print(ads.loc[ind]['refined'])
    else:
        print(row['name'])
    print('===============================')
    
    for end in replace_ends_with:
        if row['name'].endswith(end):
            ads.loc[ind]['refined'] = row['name'].replace(end, '')

ads = ads.reset_index(drop=True)

#### 전처리와 약물이 아닌 레코드를 제거하면 9250개가 남습니다

In [16]:
ads.shape

(9250, 2)

#### B. 약-부작용 관계를 나타는 것이 아닌 경우, 지웁니다

In [21]:
non_drug_ad_rel = ['-free', 'free-', ' free ', 'variable', 'wrong drug administered', 'exercise-induced',
                  'underdose', 'switch', 'massage', 'according', '-defining', 'arizona', 'controller', 'athletics',
                  'atopics', 'average', 'bachelor', 'award', '—gasoline', 'cards', 'comer', 'consisting', 'consumer',
                  'courage', 'covaria', 'coverage', 'dimers', 'distinguish', 'eavalues', 'emilia', 'multivaria',
                  'nsaids', 'preexisting', 'reaches', 'recording', 'restaura', 'spain', 'standard', 'tactics',
                  'tattoos', 'tasmania', 'telcagepant', 'testing', 'tragus', '<02'] #'non'

rep = dict((re.escape(k), ' ') for k in non_drug_ad_rel)
pattern = re.compile("|".join(rep.keys()))

In [22]:
for ind, row in ads.iterrows():
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if row['name'].strip() != refined.strip():
        print(row['name'])
        ads.drop(ind, inplace=True)
        print('===============================')

ads = ads.reset_index(drop=True)

values<02


#### 약-부작용 관계를 나타는 것이 아닌 레코드를 제거하면 9132 남습니다

In [23]:
ads.shape

(9132, 2)

### 부작용용어 dictionary와 정확이 매칭하면 추가 전처리 없이 확정

In [24]:
for ind, ad in ads.iterrows():
#     print(ad['name'].strip())
    name = ad['name'].strip()
    exact_mathing = dictionary.loc[dictionary.t_ad==name]
    
    if len(exact_mathing)==0:
        if name[0] in ['"', '.', ':', '-', '(', ')']:
            name = name[1:]
        if name[len(name)-1] in ['"', '.', ':', '-', '(', ')']:
            name = name[:len(name)-1]
#         print(name)
            
        exact_mathing = dictionary.loc[dictionary.t_ad==name]
        
    if len(exact_mathing)>0:
        ads.loc[ind]['refined'] = name

### 추가 전처리가 필요한 약물 갯수

In [25]:
len(ads[(ads.refined=='')])#.sort_values(by=['name'])[2520:2540]

2317

In [26]:
def get_only_string(text):
    try:
        float(text)
        return False
    except ValueError:
        return True
    
def remove_comparison(name, spliter):
    arrs = list(filter(lambda x: get_only_string(x.strip().replace('%', '')) and x.strip(), name.split(spliter)))
    if len(arrs)>0:
        return arrs
    else:
        return None

for ind, row in ads.loc[ads.name.str.find('<')>=0].iterrows():
    processed = remove_comparison(row['name'], '<')
    if processed is None:
        ads.drop(ind, inplace=True)
    else:
        ads.loc[ind].refined = ''.join(processed)
# comp_ads.refined = comp_ads.name.map(lambda x: remove_comparison(x))

In [27]:
ads.loc[ads.name.str.find('<')>=0]

,name,refined
647,values<0.2,values
757,stenosis<50,stenosis
1113,<asc-h,asc-h
2148,pressure<80,pressure
2862,thrombocytopenia<50,thrombocytopenia
3123,diabetes<5,diabetes
3286,values<15,values
4274,values<00001,values
4596,binge drinking<1,binge drinking
4715,stenosis<70%,stenosis


여러 부작용이 하나의 레코드에 담겨있는 경우, 분리하여 입력

In [28]:
additional_ads = []

for ind, row in ads.loc[ads.name.str.find('>')>=0].iterrows():
    processed = remove_comparison(row['name'], '>')
    if processed is None:
        ads.drop(ind, inplace=True)
    else:
        ads.loc[ind].refined = processed[0]
        for pro in processed[1:]:
            print(pro)
            additional_ads.append({'name':row['name'], 'refined':pro})

ads = ads.append(pd.DataFrame(additional_ads))
ads.drop_duplicates(inplace=True)

transient ischaemic attack
amaurosis
retinal infarction
transient
healthy
retinal
amaurosis
transient
healthy
amaurosis fugax


In [29]:
ads.shape

(9141, 2)

### 앞 뒤의 -, . 제거

In [30]:
remove_list = ['.', '-related', '-', ':', '+', '–', ';', '%', 'time', 'invariably', '||']
def preprocess(name):
    if name[0] in remove_list:
        name = name[1:len(name)]
    
    if name[len(name)-1] in remove_list:
        name = name[0:len(name)-1]
    return name
    
ads.name = ads.name.map(lambda x: preprocess(x))

In [31]:
ads.sort_values(by='name').head()

,name,refined
6732,(arrhythmia),arrhythmia
7444,(copd),copd
3480,(lbbb),lbbb
8763,(scale,scale
7985,(systemic,systemic


### 추가 전처리 필요없는 부작용들은  refined로 입력

In [ ]:
remove_list.extend([')', '('])
spe_chars = dict((re.escape(str(k)), ' ') for k in remove_list)
sep_pattern = re.compile("|".join(spe_chars.keys()))
int_pattern = re.compile('[0-9]')

for ind, row in ads.iterrows():
    refined = sep_pattern.sub(lambda m: spe_chars[re.escape(m.group(0))], row['name'])
    int_refined = int_pattern.search(row['name'])
    
    if refined.strip() == '':
        ads.drop(ind, inplace=True)
    elif row['name'].strip() == refined.strip() and int_refined is None:
        ads.loc[ind]['refined'] = refined
        print(row['name'])
        print(ads.loc[ind]['refined'])
    print('===============================')

In [33]:
ads.loc[ads.refined.str.len()>0].shape

(8388, 2)

In [34]:
ads.reset_index(drop=True)

,name,refined
0,anuloaortic ectasia,
1,mass-cavity,
2,presyncope,presyncope
3,seizure,seizure
4,aphthous ulcer,aphthous ulcer
5,proliferative diabetic retinopathy,proliferative diabetic retinopathy
6,post-aids,
7,shoulder pain,shoulder pain
8,granuloma,granuloma
9,gallbladder stones,gallbladder stones


## 정체된 목록입니다.

In [35]:
ads.loc[ads.refined.str.strip()!=''].to_csv('./refined_ads.csv', sep=',')

## 선생님께 확인 요청을 드리고 싶은 부분

In [37]:
# - 표시
ads[(ads['refined']=='') & (ads['name'].str.find('-')>=0)].shape

(690, 2)

In [38]:
ads[(ads['refined']=='') & (ads['name'].str.find('-')>=0)]

,name,refined
1,mass-cavity,
6,post-aids,
14,epispadias-classical,
24,broncho-pneumonia,
40,cardio-myopathy,
50,hypertension-stroke,
64,mgh-adhd,
77,heat-shock,
86,neoplasm-associated,
114,neoangiogenesis-inflammation,


In [41]:
ads[ads.name.str.find("nstemi")>=0] 

,name,refined
3749,riddle-nstemi,
4024,nstemi,nstemi
6817,lipsia-nstemi,
6845,nstemi,nstemi


### fluorodeoxyglucose 같은 용어들

In [42]:
ads[ads.name.str.find('fluoro')>=0]

,name,refined
136,immunofluorometric,immunofluorometric
265,fluoroscopy,fluoroscopy
642,5-fluorouracil,
867,videofluorography,videofluorography
1606,fluorouracil-based,
1996,fluoropyrimidine,fluoropyrimidine
3017,fluorouracil,fluorouracil
3295,f-18-fluorodeoxyglucose,
3683,fluorodeoxyglucose,fluorodeoxyglucose
3867,polytetrafluoroethylene,polytetrafluoroethylene


### in situ 포함 부작용

In [43]:
ads[ads.name.str.find('in situ')>=0]

,name,refined
889,melanoma in situ,melanoma in situ
4296,adenocarcinoma in situ,adenocarcinoma in situ
4959,ductal carcinoma in situ,ductal carcinoma in situ
6620,in situ cancers,in situ cancers
7892,carcinoma in situ,carcinoma in situ


- 추가적으로 정제하는데 어려움을 겪고 있는 부작용 용어가 757개 정도 됩니다. 한번 봐주시면서 어떤 접근 방식이 좋을 지 알려주시면 감사하겠습니다.

In [44]:
ads[ads['refined'] == ''].shape

(752, 2)

In [45]:
ads[ads['refined'] == ''].to_csv('./need_to_refined_ads.csv', sep=',')